In [1]:
import sys
import os
import nbimporter
import copy
import math
import numpy
import json
from collections import OrderedDict
import pprint

# 外皮入力情報から簡易熱負荷計算入力情報へのコンバート

### 外皮入力情報補完

#### Outline

入力情報レベルに応じて、LV1からLV4までの外皮入力情報の補完を行う。

#### Function

In [2]:
def convert_design_house(d):
    
    # LV1からLV2
    if d['InputLevel'] == 1:
        import LV1toLv2 as mf_1_to_2
        d2 = mf_1_to_2.convert(d)
                
    # LV2からLV3
    if d['InputLevel'] < 2:     
        import LV2toLv3 as mf_2_to_3
        d3 = mf_2_to_3.convert(d2) 
    elif d['InputLevel'] == 2:
        import LV2toLv3 as mf_2_to_3
        d3 = mf_2_to_3.convert(d)    
        
    # LV3からLV4
    if d['InputLevel'] < 3:     
        import LV3toLv4 as mf_3_to_4
        d4 = mf_3_to_4.convert(d3) 
    elif d['InputLevel'] == 3:
        import LV3toLv4 as mf_3_to_4
        d4 = mf_3_to_4.convert(d) 
    elif d['InputLevel'] == 4:
        d4 = copy.deepcopy(d_excel) 
        
    return d4

### 暖冷房設備の能力等の設定

#### Outline

暖冷房設備の種類が対流式の場合は、ルームエアコンディショナーの定格能力・最大能力・定格能力を取得する。  
暖房設備が放射式の場合は、床暖房の敷設面積を取得する。

#### Function

In [3]:
def convert_equipment(mf_equipment, d):

    import equipment as mf_equipment
    
    equipment_main = {}
    if d['Common']['MainOccupantRoomFloorArea'] > 0 :
        equipment_main = mf_equipment.convert(A=d['Common']['MainOccupantRoomFloorArea'], 
                                              **d['CoolingAndHeating']['main']) 

    equipment_other = {}
    if d['Common']['OtherOccupantRoomFloorArea'] > 0 :
        equipment_other = mf_equipment.convert(A=d['Common']['OtherOccupantRoomFloorArea'],
                                               **d['CoolingAndHeating']['other']) 
        
    return equipment_main, equipment_other

### 設計住戸の熱負荷計算用入力ファイルの書き出し

#### Outline

設計住戸の熱負荷計算用ファイルの作成を行う。

#### Function

In [4]:
 def output_design_house(mf_4_to_calc, d4, equipment_main, equipment_other):
        
    d_calc = mf_4_to_calc.convert(d4, equipment_main, equipment_other) 
    
    with open('DesignHouse.json', 'w') as f_design:
        json.dump(d_calc, f_design, indent=4)

### 基準値計算用住戸の熱負荷計算用入力ファイルの書き出し

#### Outline

基準値計算用住戸の熱負荷計算用ファイルの作成を行う。

#### Function

In [5]:
def get_equipment_standard(mf_equipment, A):
    
    if A > 0 :
        q = mf_equipment.get_capacity_RAC(A)

        return {
                'cooling': {
                    'main': {
                        'type'   : 'room_air_conditioner',
                        'q_rtd_c': q['q_rtd_c'],
                        'q_max_c': q['q_max_c'],
                        'e_rtd_c': None},
                    'sub': {}
                },
                'heating': {
                    'main': {
                        'type'   : 'room_air_conditioner',
                        'q_rtd_h': q['q_rtd_h'],
                        'q_max_h': q['q_max_h'],
                        'e_rtd_h': None},
                    'sub': {}
                }
            }
    
    else :
        return{}

In [6]:
def convert_standard_house(d):
    
    import StandardHouse as mf_standard
    d_standard = mf_standard.convert(d) 

    return d_standard

In [7]:
def output_standard_house(mf_4_to_calc, mf_equipment, HouseType, area_main, area_other, d):
    
    d['Common']['HouseType'] = HouseType

    # 設備情報の取得
    equipment_standard_main  = get_equipment_standard(mf_equipment, area_main)
    equipment_standard_other = get_equipment_standard(mf_equipment, area_other)
    
    with open('StandardHouse.json', 'w') as f_standard:
        json.dump(mf_4_to_calc.convert(convert_standard_house(d), 
                                       equipment_standard_main, equipment_standard_other),
                  f_standard, indent=4), 

### 統合

#### Outline

入力情報から簡易熱負荷計算用入力ファイルを作成する。

#### Function

In [8]:
def convert_input_for_calculation(d_excel, d_web):

    # 用途別床面積情報の受け渡し
    area_main  = d_web['Common']['MainOccupantRoomFloorArea']
    area_other = d_web['Common']['OtherOccupantRoomFloorArea']
    
    d_excel['MainOccupantRoomFloorArea']  = area_main
    d_excel['OtherOccupantRoomFloorArea'] = area_other
    d_excel['TotalFloorArea']             = d_web['Common']['TotalFloorArea']
    
    # 外皮入力情報補完
    d4 = convert_design_house(d_excel)
    
    # 設備情報の取得
    import equipment as mf_equipment
    equipment_main, equipment_other = convert_equipment(mf_equipment, d_web)
    
    # 設計住戸の熱負荷計算用ファイルの書き出し
    import LV4toCalc as mf_4_to_calc
    output_design_house(mf_4_to_calc, d4, equipment_main, equipment_other)

    # 基準値計算用住戸の熱負荷計算用ファイルの書き出し
    output_standard_house(mf_4_to_calc, mf_equipment, d_web['Common']['HouseType'], area_main, area_other, d4)

In [9]:
d_excel = {
    'InputLevel': 1,
    'Region' : 6,
    'InsulationType' : 'FloorIns',
    'InsulationTypeBathroom' : 'NotInContactWithOutsideAir',
    'URoof' : 0.24,
    'UWall' : 0.53,
    'UFloor' : 0.48,
    'UFloorBathroom' : 0.48,
    'UDoor' : 4.65,
    'UW'    : 3.49,
    'EtaDC' : 0.51,       
    'EtaDH' : 0.51, 
    'IsInputFValue' : True, 
    'FValueC' : 0.9,
    'FValueH' : 0.7,
    'psi_entrance' : 1.8,
    'psi_bathroom' : 1.8
}

d_web = {
    'Common':{
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 30.0,
        'TotalFloorArea': 90.0,
        'HouseType': 'detached'
    },
    'CoolingAndHeating':{
        'main' : {
            'cooling':{
                'type': 'convection',
                'division': '（い）'
            },
            'heating':{
                'type': 'convection',
                'division': '（ろ）'
            },
        },
        'other': {
            'cooling':{
                'type': 'convection',
                'division': '（ろ）'
            },
            'heating':{
                'type': 'radiation',
                'r': 0.7
            },
        },
    }
}

convert_input_for_calculation(d_excel, d_web)

Importing Jupyter notebook from LV1toLv2.ipynb
Importing Jupyter notebook from RoundNumber.ipynb
Importing Jupyter notebook from LV2toLv3.ipynb
Importing Jupyter notebook from LV3toLv4.ipynb
Importing Jupyter notebook from equipment.ipynb
Importing Jupyter notebook from LV4toCalc.ipynb
Importing Jupyter notebook from StandardHouse.ipynb
Importing Jupyter notebook from nu.ipynb
UA基準値0.87,ηAH基準値4.3,ηAC基準値2.8
U値{'Roof': 0.2049755366143829, 'Wall': 0.45265431002342893, 'Floor': 0.4099510732287648, 'Window': 3.971401021903669, 'EarthfloorPerimeter': 0.6490891992788776},開口部η0.4386836640077342,開口部ηH0.46743989470727487,開口部ηC0.40992743330819353
UA計算値0.8699999999999998,ηAH計算値4.3,ηAC計算値2.8000000000000003
